In [ ]:
import sqlite3
import pandas as pd
import tldextract

pd.set_option('display.max_rows', 500)

In [ ]:
# HELPER FUNCTIONS
# Function to replace URLs with domain names
def replace_url_with_domain(text):
    extracted = tldextract.extract(text)
    if extracted.domain:
        return extracted.domain
    return text

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('tweets_4.db')
query = "SELECT * FROM tweets"

# Load the data into a pandas DataFrame
data = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
# Process tweet text data
df = data[data.handle != '@TrainedOnTest']
df['content'] = df['content'].str.lower()

# ignore chars
df['content'] = df['content'].str.replace("@", "")
df['content'] = df['content'].str.replace("!", "")
df['content'] = df['content'].str.replace("_", "")
df['content'] = df['content'].str.replace("install", "")
df['content'] = df['content'].str.replace("?", "")

# Stopwords
df = df[df.content.str.len() < 25]
df['content'] = df['content'].str.replace("homehomebrew", "homebrew")
df['content'] = df['content'].str.replace("home brew", "homebrew")
df['content'] = df['content'].str.replace("brew  brew", "homebrew")
df['content'] = df['content'].str.replace("homehomebrew homehomebrew", "homebrew")
df['content'] = df['content'].str.replace("instead of chrome", "")
df['content'] = df['content'].str.strip()


# fixes
# df['content'] = df['content'].str.replace("  ", ",")
df['content'] = df['content'].str.replace(" and ", ",")
df['content'] = df['content'].str.replace(" & ", ",")
df['content'] = df['content'].str.replace("\n", ",")
df['content'] = df['content'].str.replace(" + ", ",")
df['content'] = df['content'].apply(replace_url_with_domain)
df['content'] = df['content'].str.split(',')
df = df.explode('content')


df['content'] = df['content'].str.replace("this", "")
df['content'] = df['content'].str.replace("underrated", "")
df['content'] = df['content'].str.replace("+1", "")
df['content'] = df['content'].str.replace("there's only one answer", "")

# df = df[~df.content.isna()]
df = df[df.content != ""]

df.reset_index(drop=True, inplace=True)

In [ ]:
data[data.duplicated(subset=['handle', 'content'])]

In [ ]:
df.head(100)

In [ ]:
df.shape

In [ ]:
df.content.value_counts().sort_values(ascending=True).tail(50).plot(kind='barh', figsize=(15, 15), title=f"Top 50 Recommended Apps from Tweet Replies")